In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS 
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import math
import requests
import io
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',1000)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Copy of US_Accidents_June20.csv

') #,nrows=10000)
redunCol=['ID','Source','End_Lat','End_Lng','Number','Street','Zipcode','Country','Weather_Timestamp'
            ,'Airport_Code','Astronomical_Twilight','Civil_Twilight','Nautical_Twilight']
df=df.drop(redunCol,axis=1)
df

,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Side,City,County,State,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.010,Right lane blocked due to accident on I-70 Eas...,R,Dayton,Montgomery,OH,US/Eastern,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
1,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.010,Accident on Brice Rd at Tussing Rd. Expect del...,L,Reynoldsburg,Franklin,OH,US/Eastern,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
2,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident on OH-32 State Route 32 Westbound at ...,R,Williamsburg,Clermont,OH,US/Eastern,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night
3,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident on I-75 Southbound at Exits 52 52B US...,R,Dayton,Montgomery,OH,US/Eastern,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
4,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident on McEwen Rd at OH-725 Miamisburg Cen...,R,Dayton,Montgomery,OH,US/Eastern,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,NaN,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,0.543,At Market St - Accident.,R,Riverside,Riverside,CA,US/Pacific,86.0,86.0,40.0,28.92,10.0,W,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
3513613,NaN,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,0.338,At Camino Del Rio/Mission Center Rd - Accident.,R,San Diego,San Diego,CA,US/Pacific,70.0,70.0,73.0,29.39,10.0,SW,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
3513614,NaN,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,0.561,At Glassell St/Grand Ave - Accident. in the ri...,R,Orange,Orange,CA,US/Pacific,73.0,73.0,64.0,29.74,10.0,SSW,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Day
3513615,NaN,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,R,Culver City,Los Angeles,CA,US/Pacific,71.0,71.0,81.0,29.62,10.0,SW,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day


## Data Cleaning

In [ ]:
redunCol=['TMC','Start_Time','End_Time','Description','City','County','State','Timezone','Distance(mi)','Wind_Direction']
df=df.drop(redunCol,axis=1)

In [ ]:
df.loc[df['Side']=='R','Side']=1 #Right side is 1
df.loc[df['Side']=='L','Side']=0 #Left side is 
df.loc[df['Sunrise_Sunset']=='Day','Sunrise_Sunset']=0 #daytime is 0
df.loc[df['Sunrise_Sunset']=='Night','Sunrise_Sunset']=1 #nighttime is 1
df

,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,3,39.865147,-84.058723,1,36.9,NaN,91.0,29.68,10.0,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1
1,2,39.928059,-82.831184,0,37.9,NaN,100.0,29.65,10.0,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1
2,2,39.063148,-84.032608,1,36.0,33.3,100.0,29.67,10.0,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,1
3,3,39.747753,-84.205582,1,35.1,31.0,96.0,29.64,9.0,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,1
4,2,39.627781,-84.188354,1,36.0,33.3,89.0,29.65,6.0,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,2,34.002480,-117.379360,1,86.0,86.0,40.0,28.92,10.0,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0
3513613,2,32.766960,-117.148060,1,70.0,70.0,73.0,29.39,10.0,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0
3513614,2,33.775450,-117.847790,1,73.0,73.0,64.0,29.74,10.0,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,0
3513615,2,33.992460,-118.403020,1,71.0,71.0,81.0,29.62,10.0,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0


In [ ]:
cols=df['Weather_Condition'].unique()
for col in cols:
    df[col] = df['Weather_Condition'] == col

In [ ]:
del(df['Wind_Chill(F)'])
df["Precipitation(in)"].fillna(0, inplace = True)
df['Wind_Speed(mph)'].fillna((df['Wind_Speed(mph)'].mean()), inplace = False)
df

,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
0,3,39.865147,-84.058723,1,36.9,91.0,29.68,10.0,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,39.928059,-82.831184,0,37.9,100.0,29.65,10.0,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,39.063148,-84.032608,1,36.0,100.0,29.67,10.0,3.5,0.00,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,3,39.747753,-84.205582,1,35.1,96.0,29.64,9.0,4.6,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2,39.627781,-84.188354,1,36.0,89.0,29.65,6.0,3.5,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,2,34.002480,-117.379360,1,86.0,40.0,28.92,10.0,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,2,32.766960,-117.148060,1,70.0,73.0,29.39,10.0,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,2,33.775450,-117.847790,1,73.0,64.0,29.74,10.0,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,2,33.992460,-118.403020,1,71.0,81.0,29.62,10.0,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df = df.dropna()
del(df['Weather_Condition'])

In [ ]:
df=df.sample(n=1000000, random_state=1)

In [ ]:
df

,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
694610,2,41.214500,-111.978989,0,24.0,71.0,25.29,5.0,14.0,0.00,False,False,True,False,False,False,False,False,False,True,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1036014,2,35.791061,-78.651611,1,79.0,88.0,29.66,10.0,0.0,0.00,False,False,True,False,False,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2602907,2,34.440420,-118.599810,1,64.0,11.0,29.11,10.0,10.0,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,1,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3208166,2,40.544130,-111.951887,0,32.2,68.0,30.28,10.0,3.5,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2092667,3,33.691933,-117.337677,1,48.2,46.0,30.14,10.0,5.8,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121417,2,38.547530,-112.373840,1,38.0,79.0,25.24,10.0,3.0,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2069743,4,42.291107,-83.838654,1,21.9,85.0,30.15,1.5,10.4,0.01,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2230268,3,32.675133,-97.006790,1,91.4,18.0,29.83,10.0,12.7,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3234174,4,29.603440,-95.202790,1,80.1,81.0,29.85,10.0,6.9,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Splitting into features and severity

In [ ]:
y = df['Severity'].copy()
y

694610     2
1036014    2
2602907    2
3208166    2
2092667    3
          ..
3121417    2
2069743    4
2230268    3
3234174    4
1400137    2
Name: Severity, Length: 1000000, dtype: int64

In [ ]:
x = df.drop(['Severity'], axis = 1)
x

,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,Overcast,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
694610,41.214500,-111.978989,0,24.0,71.0,25.29,5.0,14.0,0.00,False,False,True,False,False,False,False,False,False,True,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1036014,35.791061,-78.651611,1,79.0,88.0,29.66,10.0,0.0,0.00,False,False,True,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2602907,34.440420,-118.599810,1,64.0,11.0,29.11,10.0,10.0,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3208166,40.544130,-111.951887,0,32.2,68.0,30.28,10.0,3.5,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2092667,33.691933,-117.337677,1,48.2,46.0,30.14,10.0,5.8,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121417,38.547530,-112.373840,1,38.0,79.0,25.24,10.0,3.0,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2069743,42.291107,-83.838654,1,21.9,85.0,30.15,1.5,10.4,0.01,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2230268,32.675133,-97.006790,1,91.4,18.0,29.83,10.0,12.7,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3234174,29.603440,-95.202790,1,80.1,81.0,29.85,10.0,6.9,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Splitting into tranning, test and validation datasets

In [ ]:
x_train1, x_test, y_train1, y_test = train_test_split(x,y,test_size=0.1, random_state=42, shuffle=True)

In [ ]:
x_train, x_validation, y_train, y_validation = train_test_split(x_train1,y_train1,test_size=0.167, random_state=42, shuffle=True)

In [ ]:
mask1 = (df['Severity'] == 1)
mask2 = (df['Severity'] == 2)
mask3 = (df['Severity'] == 3)
mask4 = (df['Severity'] == 4)

ratio2 = len(df[mask2])/(len(df[mask1])+len(df[mask2]))
ratio3 = len(df[mask3])/(len(df[mask3])+len(df[mask4]))

print(ratio2,ratio3)

0.9864795633287907 0.900404863507295


## Data Modelling using Decision Tree Classifier

In [ ]:
max = 0
imax = 0
for i in range(5,100,5):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 67.90219560878243 i: 5
acc: 67.90219560878243 i: 10
acc: 69.18429807052561 i: 15
acc: 69.228875582169 i: 20
acc: 69.46373918829009 i: 25
acc: 69.57618097139056 i: 30
acc: 69.8236859614105 i: 35
acc: 69.93280106453759 i: 40
acc: 70.25149700598803 i: 45
acc: 70.54224883566201 i: 50
acc: 70.9767132401863 i: 55
acc: 71.2022621423819 i: 60
acc: 71.2022621423819 i: 65
acc: 71.26147704590818 i: 70
acc: 71.30405854956753 i: 75
acc: 71.36793080505656 i: 80
acc: 71.36793080505656 i: 85
acc: 71.35395874916833 i: 90
acc: 71.39587491683301 i: 95


In [ ]:
for i in range(100,200,10):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 71.51696606786427 i: 100
acc: 71.55821689953427 i: 110
acc: 71.76114437791085 i: 120
acc: 71.87358616101132 i: 130
acc: 71.938123752495 i: 140
acc: 71.99268130405855 i: 150
acc: 72.0971390552229 i: 160
acc: 72.45575515635396 i: 170
acc: 72.46773120425814 i: 180
acc: 72.63140385894877 i: 190


In [ ]:
for i in range(200,300,10):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 72.70192947438457 i: 200
acc: 72.79574184963407 i: 210
acc: 73.05455755156353 i: 220
acc: 73.17365269461078 i: 230
acc: 73.25083166999335 i: 240
acc: 73.28476380572188 i: 250
acc: 73.36992681304059 i: 260
acc: 73.43912175648703 i: 270
acc: 73.48436460412509 i: 280
acc: 73.48702594810379 i: 290


In [ ]:
for i in range(300,400,20):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 73.55688622754492 i: 300
acc: 73.79973386560212 i: 320
acc: 73.9148369926813 i: 340
acc: 73.96074517631403 i: 360
acc: 74.04590818363273 i: 380


In [ ]:
for i in range(400,1000,50):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 74.06786427145708 i: 400
acc: 74.3725881570193 i: 450
acc: 74.65535595475716 i: 500
acc: 74.78642714570857 i: 550
acc: 74.89021956087825 i: 600
acc: 75.10711909514305 i: 650
acc: 75.32002661343978 i: 700
acc: 75.5249500998004 i: 750
acc: 75.68995342648037 i: 800
acc: 75.81769793745842 i: 850
acc: 75.89288090485695 i: 900
acc: 75.96540252827678 i: 950


In [ ]:
for i in range(1000,10000,1000):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 76.06986027944112 i: 1000
acc: 77.55355954757152 i: 2000
acc: 78.24417831004658 i: 3000
acc: 78.60545575515636 i: 4000
acc: 78.87225548902195 i: 5000
acc: 79.01996007984032 i: 6000
acc: 79.0585495675316 i: 7000
acc: 79.07518296739853 i: 8000
acc: 79.05389221556887 i: 9000


In [ ]:
for i in range(10000,15000,1000):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 79.07451763140386 i: 10000
acc: 79.10778443113773 i: 11000
acc: 79.0645375914837 i: 12000
acc: 79.0172987358616 i: 13000
acc: 79.02129075182968 i: 14000


In [ ]:
for i in range(15000,20000,1000):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 79.0086493679308 i: 15000
acc: 78.98735861610113 i: 16000
acc: 78.96872920825017 i: 17000
acc: 78.94610778443113 i: 18000
acc: 78.88955422488357 i: 19000


In [ ]:
for i in range(20000,30000,1000):
    severity_classifier = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    severity_classifier.fit(x_train,y_train)
    y_valid_pred = severity_classifier.predict(x_validation)
    t = accuracy_score(y_valid_pred,y_validation)*100
    print("acc:",t,"i:",i)
    if(t>max):
        max = t
        imax = i

acc: 78.85961410512309 i: 20000
acc: 78.80172987358615 i: 21000
acc: 78.77312042581504 i: 22000
acc: 78.7272122421823 i: 23000
acc: 78.69261477045907 i: 24000
acc: 78.62940785096474 i: 25000
acc: 78.583499667332 i: 26000
acc: 78.49500998003992 i: 27000
acc: 78.44111776447106 i: 28000
acc: 78.36859614105123 i: 29000


## Final Validation score

In [ ]:
severity_classifier = DecisionTreeClassifier(max_leaf_nodes=imax, random_state=0)
severity_classifier.fit(x_train,y_train)
y_valid_pred = severity_classifier.predict(x_validation)
t = accuracy_score(y_valid_pred,y_validation)*100
t

74.04590818363273

## Test Score

In [ ]:
severity_classifier = DecisionTreeClassifier(max_leaf_nodes=imax, random_state=0)
severity_classifier.fit(x_train,y_train)
y_pred = severity_classifier.predict(x_test)
t = accuracy_score(y_pred,y_test)*100

In [ ]:
t

78.737

In [ ]:
imax

11000

## Classification Matrix

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           1      0.479     0.143     0.220       937
           2      0.840     0.871     0.855     67959
           3      0.671     0.682     0.676     28038
           4      0.392     0.104     0.165      3066

    accuracy                          0.787    100000
   macro avg      0.595     0.450     0.479    100000
weighted avg      0.776     0.787     0.778    100000

